# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import csv
import json
from pandas.io.json import json_normalize
from pprint import pprint

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
csv_path = "/Users/khushbooshah/Desktop/My work/Homework/python-api-challenge/VacationPy/output_data2/weatherdata2.csv"
weather_data = pd.read_csv(csv_path)

weather_data.head()

,CityID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max temp,Wind Speed
0,0,Parrita,75,CR,1580260753,94,9.52,-84.32,68.00,3.36
1,1,Pangnirtung,9,CA,1580260753,84,66.15,-65.71,-26.73,3.31
2,2,Constitucion,30,CL,1580260753,68,-35.33,-72.42,58.51,16.02
3,3,Saint-Philippe,90,RE,1580260753,64,-21.36,55.77,73.40,8.05
4,4,Jamestown,90,US,1580260754,92,42.10,-79.24,32.00,6.93


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)
locations=weather_data[["Lat","Lng"]].astype(float)
heat=weather_data["Humidity"].astype(float)
maxhumid=weather_data["Humidity"].max()

myheatmap = gmaps.figure()
HeatLayer=gmaps.heatmap_layer(locations,weights=heat,max_intensity=maxhumid,point_radius=1,dissipating=False)
myheatmap.add_layer(HeatLayer)
myheatmap

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
NewData=weather_data.dropna(how="all")
NewData=NewData.loc[(NewData["Cloudiness"] == 0) & (NewData["Wind Speed"] < 10) & ((NewData["Max temp"] > 70) & (NewData["Max temp"] <= 80))]
len(NewData)

5

In [6]:
hotel_df=NewData
NewData

,CityID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max temp,Wind Speed
5,5,Tomatlan,0,MX,1580260756,61,19.93,-105.25,74.71,3.71
175,175,Chiredzi,0,ZW,1580261321,71,-21.05,31.67,70.57,1.16
243,243,Rock Sound,0,BS,1580261331,73,24.90,-76.20,74.79,8.70
445,445,San Patricio,0,MX,1580261360,64,19.22,-104.70,76.57,5.91
527,527,Capitan Bado,0,PY,1580261372,65,-23.27,-55.53,76.84,6.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
HName = []
HAddress=[]
# function to extract hotel name
def HotelName(Lat,Lng):
    Radius = 5000
    Type = "Hotel"
    Coordinates =  Lat + " , " + Lng 
    params = {
    "location": Coordinates,
    "radius": Radius,
    "type": Type,
    "key": g_key
    }
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    resp = requests.get(base_url, params=params)
    rtext = json.loads(resp.text)
    response = json_normalize(rtext)
    HotelResults=json_normalize(response["results"][0])
    HName.append(HotelResults["name"][1])
    HAddress.append(HotelResults["vicinity"][1])
    
    

In [8]:
for index, row in NewData.iterrows():
    HotelName(str(row['Lat']),str(row['Lng']))

In [9]:
HName,HAddress

(['Hospital de Primer Contacto Tomatlan',
  'ZETDC',
  'Market Place Store',
  'Hotel Cabo Blanco',
  'EE CEL. Sapucaia'],
 ['Independencia, Las Primaveras, Tomatlán',
  'Chiredzi',
  'Meridian Highway, Rock Sound',
  'Armada y, Puerto de la Navidad SN Fracc, Fracc, Barra de Navidad',
  'Rua Fortunato de Oliveira, 150, Coronel Sapucaia'])

In [10]:
hotel_df.insert(2, "Hotel Name", HName, True) 
hotel_df.insert(3, "Hotel Address", HAddress, True)
hotel_df

,CityID,City,Hotel Name,Hotel Address,Cloudiness,Country,Date,Humidity,Lat,Lng,Max temp,Wind Speed
5,5,Tomatlan,Hospital de Primer Contacto Tomatlan,"Independencia, Las Primaveras, Tomatlán",0,MX,1580260756,61,19.93,-105.25,74.71,3.71
175,175,Chiredzi,ZETDC,Chiredzi,0,ZW,1580261321,71,-21.05,31.67,70.57,1.16
243,243,Rock Sound,Market Place Store,"Meridian Highway, Rock Sound",0,BS,1580261331,73,24.90,-76.20,74.79,8.70
445,445,San Patricio,Hotel Cabo Blanco,"Armada y, Puerto de la Navidad SN Fracc, Fracc...",0,MX,1580261360,64,19.22,-104.70,76.57,5.91
527,527,Capitan Bado,EE CEL. Sapucaia,"Rua Fortunato de Oliveira, 150, Coronel Sapucaia",0,PY,1580261372,65,-23.27,-55.53,76.84,6.82


In [11]:
# Create a Hotels
# Store 'Lat' and 'Lng' into  locations 
HLocations = hotel_df[["Lat", "Lng"]].astype(float)
HumidityValue = weather_data["Humidity"].astype(float)
HumidityMax=weather_data["Humidity"].max()

HotelLocator = gmaps.figure()

HHLayer = gmaps.heatmap_layer(locations, weights=HumidityValue, 
                                 dissipating=False, max_intensity=HumidityMax,
                                 point_radius = 1)

HotelLocator.add_layer(HHLayer)
HotelLocator

Figure(layout=FigureLayout(height='420px'))

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
InfoTemplate = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name


In [13]:
# Add marker layer ontop of heat map
# Create a Hotels
# Store 'Lat' and 'Lng' into  locations 
#info_box_template = """


# Display Map

In [14]:
HoverMarker=np.array(hotel_df["City"])
HotelMarker_Layer=gmaps.marker_layer(HLocations,info_box_content=InfoTemplate,display_info_box=True)
HotelLocator.add_layer(HotelMarker_Layer)


# Display Map
HotelLocator

Figure(layout=FigureLayout(height='420px'))